In [1]:
field_easy = [ # 0 = empty, 1 = food, 2 = wall, 3 = pacman
  [3, 1, 1, 1],
  [0, 2, 2, 1],
  [0, 1, 2, 1],
  [0, 0, 0, 1],
]

field_medium = [
  [3, 1, 1, 2, 1],
  [0, 2, 1, 2, 1],
  [1, 2, 1, 1, 1],
  [1, 1, 2, 2, 1],
  [0, 1, 1, 1, 1],
]

field_hard = [
  [3, 2, 1, 1, 2, 1],
  [1, 2, 0, 2, 2, 1],
  [1, 1, 1, 0, 1, 1],
  [2, 2, 1, 2, 1, 2],
  [1, 0, 1, 1, 1, 1],
  [1, 2, 2, 2, 2, 1],
]

field_very_hard = [
  [3, 2, 1, 2, 1, 2, 1],
  [1, 2, 0, 2, 0, 2, 1],
  [1, 1, 1, 1, 1, 2, 1],
  [2, 2, 2, 2, 1, 2, 1],
  [1, 0, 0, 1, 1, 1, 1],
  [1, 2, 2, 2, 2, 2, 1],
  [1, 1, 1, 1, 1, 1, 1],
]

fields = [field_easy, field_medium, field_hard, field_very_hard]  

# Definição de classes

In [2]:
class PacMan:
  def __init__(self, field, x_pos=0, y_pos=0, direction='right'):
    self.x_pos = x_pos
    self.y_pos = y_pos
    self.direction = direction
    self.field = field

  def move(self):
    self.field[self.y_pos][self.x_pos] = 0

    if self.direction == 'right' and self.x_pos + 1 < len(self.field[0]):
        if self.field[self.y_pos][self.x_pos + 1] != 2:
            self.x_pos += 1
    elif self.direction == 'left' and self.x_pos - 1 >= 0 :
        if self.field[self.y_pos][self.x_pos - 1] != 2:
            self.x_pos -= 1
    elif self.direction == 'up' and self.y_pos - 1 >= 0 :
        if self.field[self.y_pos - 1][self.x_pos] != 2:
            self.y_pos -= 1
    elif self.direction == 'down' and self.y_pos + 1 < len(self.field):
        if self.field[self.y_pos + 1][self.x_pos] != 2:
            self.y_pos += 1

    has_food = self.field[self.y_pos][self.x_pos] == 1

    self.field[self.y_pos][self.x_pos] = 3

    return has_food

  def __str__(self):
    return f"PacMan is at ({self.x_pos}, {self.y_pos})"

class Node:
  def __init__(self, field, father, x_pos, y_pos):
    self.field = field
    self.father = father
    self.x_pos = x_pos
    self.y_pos = y_pos
    
  def generate_next_nodes(self):
    next_nodes = []
    for direction in ['right', 'left', 'up', 'down']:
      new_field = [row[:] for row in self.field]
      current_pos = self.x_pos, self.y_pos
      pacman = PacMan(new_field, self.x_pos, self.y_pos, direction)
      has_food = pacman.move()

      if pacman.x_pos == current_pos[0] and pacman.y_pos == current_pos[1]:
        continue

      new_field = pacman.field
      if has_food:
         next_nodes.insert(0, Node(new_field, self, pacman.x_pos, pacman.y_pos))
      else:
         next_nodes.append(Node(new_field, self, pacman.x_pos, pacman.y_pos))
    return next_nodes

  def am_i_goal(self):
    for row in self.field:
      if 1 in row:
        return False
    return True

  def __str__(self):
    return f"({self.x_pos}, {self.y_pos})"

    

# Teste unitário de geração de novos nodos

In [38]:
field = [ # 0 = empty, 1 = food, 2 = wall, 3 = pacman
  [3, 1, 1, 1],
  [0, 2, 2, 1],
  [0, 1, 2, 1],
  [0, 0, 0, 1],
]

pacman = PacMan(field, 0, 0, 'right')

node1 = Node(field, None, pacman.x_pos, pacman.y_pos)
next_nodes = node1.generate_next_nodes()

for node in next_nodes:
  print(node)  


Node at (1, 0)
Node at (0, 0)
Node at (0, 0)
Node at (0, 1)


# Teste de movimentação do PacMan

In [ ]:
ok = True
pacman = PacMan(field=field)

while ok:
  print("Current field:")
  for row in field:
    print(row)

  command = input("Enter command (up, down, left, right) or 'exit' to quit: ").strip().lower()

  if command == 'exit':
    ok = False
  elif command in ['up', 'down', 'left', 'right']:
    pacman.direction = command
    pacman.move()
    print(pacman)
  else:
    print("Invalid command. Please try again.")

Current field:
[3, 1, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (1, 0)
Current field:
[0, 3, 1, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (2, 0)
Current field:
[0, 0, 3, 1]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 3]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]
PacMan is at (3, 0)
Current field:
[0, 0, 0, 0]
[0, 2, 2, 1]
[0, 1, 2, 1]
[0, 0, 0, 1]


# Busca em largura

In [24]:
def busca_em_largura(field, x_init, y_init, debug=False):
  iterations = 0
  current_nodes = [Node(field, None, x_init, y_init)]

  while len(current_nodes) > 0:
    this_node = current_nodes.pop(0)
    if this_node.am_i_goal():
      return this_node, iterations

    next_nodes = this_node.generate_next_nodes()
    current_nodes.extend(next_nodes)

    if debug:
      for node in current_nodes:
        print(node)
      print('---------------')

    iterations += 1


In [26]:
def pac_man_com_busca_em_largura(field, debug=False):
  nodo_final, iterações = busca_em_largura(field, 0, 0, debug)

  print(f"Number of iterations: {iterações}\n")

  # Print todos os nodos pais até o nodo inicial
  nodo_atual = nodo_final
  while nodo_atual.father is not None:
    this_field = nodo_atual.field
    if debug:
      for row in this_field:
        print(row)
      print("\n")
    nodo_atual = nodo_atual.father
  if debug:
    print(nodo_atual) # nodo inicial
    print("Final node reached")
  
  return iterações

In [28]:
pac_man_com_busca_em_largura(field_medium)

Number of iterations: 854691



854691

In [11]:
iterações = []

for index, field in enumerate(fields):
  iterações.append(pac_man_com_busca_em_largura(field, debug=False))
print(f"Média: {sum(iterações)/len(iterações)}")

Number of iterations: 804



KeyboardInterrupt: 

# Busca em profundidade